In [1]:
# install packages from requirements.txt
!pip install -r requirements.txt

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\tpeiq\appdata\local\temp\pip-req-build-y2y0n_t4
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 74f9fe43758b5e5b79b2f269ce80bb0a54e72582
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached yfinance-0.2.63-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached tigeropen-3.4.0-py3-none-any.whl
  Using cached stable_baselines3-2.6.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached torch-2.7.1-cp310-cp310-win_amd64.whl.metadata (28 kB)
  Using cached scikit_learn-1.7.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached matplotlib-3.10.3-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached swig-4

  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\tpeiq\AppData\Local\Temp\pip-req-build-y2y0n_t4'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\tpeiq\AppData\Local\Temp\pip-install-deq_s3xx\elegantrl_1ebbcfce414441c1844378d2515f1c03'
  DEPRECATION: Building 'elegantrl' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'elegantrl'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
# SMART LIVE TRADING BOT SCRIPT (DAILY)
# Supports multi-stock, multi-agent SMART model, and Tiger Brokers API with balance checks, logging, and capital-aware execution

import pandas as pd
import numpy as np
import yfinance as yf
import logging
import itertools
import os
from datetime import datetime, timedelta
from stable_baselines3 import PPO, A2C, SAC

# === Setup Logging ===
date_str = datetime.now().strftime('%Y-%m-%d')
log_path = f'./log/smart_trading_bot_{date_str}.log'
error_log_path = f'./log/smart_trading_bot_error_{date_str}.log'
logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
# Improved logging: log INFO to 'smart_trading_bot.log', ERROR to 'smart_trading_bot_error.log'
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# File handler for INFO and above
info_handler = logging.FileHandler(log_path)
info_handler.setLevel(logging.INFO)
info_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
info_handler.setFormatter(info_formatter)

# File handler for ERROR and above
error_handler = logging.FileHandler(error_log_path)
error_handler.setLevel(logging.ERROR)
error_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
error_handler.setFormatter(error_formatter)

# Remove default handlers and add our handlers
if logger.hasHandlers():
    logger.handlers.clear()
logger.addHandler(info_handler)
logger.addHandler(error_handler)

logging.info("SMART Trading Bot started.")



In [3]:
import os
import tigeropen
print(os.listdir(os.path.dirname(tigeropen.__file__)))

['common', 'examples', 'fundamental', 'push', 'quote', 'tiger_open_client.py', 'tiger_open_config.py', 'trade', '__init__.py', '__pycache__']


In [4]:
# === Tiger Brokers API ===
from tigeropen.tiger_open_config import TigerOpenClientConfig
from tigeropen.common.consts import Language, OrderType
from tigeropen.trade.trade_client import TradeClient

# Fallback enum for TimeInForce if missing
class TimeInForce:
    DAY = 'DAY'
    GTC = 'GTC'
    IOC = 'IOC'

class AccountType:
    INDIVIDUAL = 'INDIVIDUAL'
    JOINT = 'JOINT'
    CORPORATE = 'CORPORATE'

# === Your trained DRL models ===
model_ppo = PPO.load("trained_models/agent_ppo")
model_a2c = A2C.load('trained_models/agent_a2c')
model_sac = SAC.load("trained_models/agent_sac")

# === Stocks to monitor ===
tickers = ['aapl', 'amd', 'amzn', 'cat', 'crwd', 'googl', 'gs', 'hd', 'ibm',
       'intc', 'meta', 'msft', 'nvda', 'pypl', 't', 'tsla', 'v']

# === Parameters ===
hmax = 100  # Max shares per trade per asset
TRADE_END_DATE = datetime.today().strftime("%Y-%m-%d")
TRAIN_START_DATE = (datetime.today() - timedelta(days=365*4)).strftime("%Y-%m-%d")
daily_loss_threshold = 0.05
capital_limit_pct = 0.75
print(TRADE_END_DATE)
print(TRAIN_START_DATE)


2025-06-24
2021-06-25


In [8]:
# === Preprocess with YahooDownloader and FeatureEngineer ===
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.config import INDICATORS

df_raw = YahooDownloader(
    start_date=TRAIN_START_DATE,
    end_date=TRADE_END_DATE,
    ticker_list=tickers
).fetch_data()

fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_vix=True,
    use_turbulence=True,
    user_defined_feature=False
)
processed = fe.preprocess_data(df_raw)

# Align to all dates and tickers
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(), processed['date'].max()).astype(str))
combination = list(itertools.product(list_date, list_ticker))
processed_full = pd.DataFrame(combination, columns=["date", "tic"]) \
    .merge(processed, on=["date", "tic"], how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])
processed_full = processed_full.fillna(0)

# Make sure 'date' column is in datetime format
processed_full['date'] = pd.to_datetime(processed_full['date'])

# Sort first for consistency
processed_full = processed_full.sort_values(by=['date', 'tic']).reset_index(drop=True)

# Assign the same index to all rows with the same date
processed_full.index = processed_full.groupby('date').ngroup()



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (17034, 8)


[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (1001, 8)
Successfully added vix


Successfully added turbulence index


In [9]:
print(processed_full.date.max())

2025-06-20 00:00:00


In [10]:
# === Build environment ===
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv 

stock_dimension = len(tickers)
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

env = StockTradingEnv(df=processed_full, **env_kwargs)
obs, _ = env.reset()
initial_account_value = env.initial_amount



Stock Dimension: 17, State Space: 171


In [11]:
print("Model observation space:", model_ppo.observation_space.shape)

Model observation space: (171,)


In [12]:
# === SMART Logic (Rolling Sharpe) ===
def calculate_sharpe(df_account_value):
    returns = df_account_value['account_value'].pct_change().dropna()
    if returns.std() == 0:
        return 0
    return (252**0.5) * returns.mean() / returns.std()

def compute_rolling_sharpes(account_value_dfs, window=30):
    sharpes = {}
    for name, df in account_value_dfs.items():
        returns = df['account_value'].pct_change().dropna()
        rolling = returns.rolling(window=window)
        rolling_sharpe = (rolling.mean() / rolling.std()) * np.sqrt(252)
        sharpes[name] = rolling_sharpe.fillna(0).values
    return sharpes

def pick_best_model(rolling_sharpes, t):
    best_model = max(rolling_sharpes, key=lambda name: rolling_sharpes[name][t] if t < len(rolling_sharpes[name]) else -np.inf)
    return best_model

# Simulate models for updated rolling Sharpe
from finrl.agents.stablebaselines3.models import DRLAgent
agent = DRLAgent(env=env)

df_account_value_a2c, df_actions_a2c = agent.DRL_prediction(model=model_a2c, environment=env)
df_account_value_sac, df_actions_sac = agent.DRL_prediction(model=model_sac, environment=env)
df_account_value_ppo, df_actions_ppo = agent.DRL_prediction(model=model_ppo, environment=env)

# Evaluate and save account value

df_account_value_a2c.to_csv("a2c_account_value.csv", index=False)
df_account_value_sac.to_csv("sac_account_value.csv", index=False)
df_account_value_ppo.to_csv("ppo_account_value.csv", index=False)



hit end!
hit end!
hit end!


In [13]:
# === Live Sharpe Tracking ===
live_returns = pd.Series(env.asset_memory).pct_change().dropna()
if live_returns.std() != 0:
    live_sharpe = (live_returns.mean() / live_returns.std()) * np.sqrt(252)
    logging.info(f"Live Sharpe ratio: {live_sharpe:.3f}")
else:
    logging.info("Live Sharpe ratio undefined (std=0)")

# === Stop-Loss Check ===
latest_value = env.asset_memory[-1]
daily_loss = (initial_account_value - latest_value) / initial_account_value
if daily_loss > daily_loss_threshold:
    logging.warning(f"Daily loss exceeded threshold ({daily_loss:.2%}). Stopping execution.")
    print("Daily stop-loss triggered. No trades executed.")
    exit()

account_value_dfs = {
    "a2c": df_account_value_a2c,
    "ppo": df_account_value_ppo,
    "sac": df_account_value_sac,
}

rolling_sharpes = compute_rolling_sharpes(account_value_dfs, window=30)
t = -1  # use last timestep
best_model = pick_best_model(rolling_sharpes, t)

if best_model == "ppo":
    final_model = model_ppo
elif best_model == "sac":
    final_model = model_sac
else:
    final_model = model_a2c

logging.info(f"Selected best model: {best_model}")
print(f"Selected best model: {best_model}")



Selected best model: ppo


In [14]:
# Display the attributes and methods of TigerOpenClientConfig
print(dir(TigerOpenClientConfig))

['_TigerOpenClientConfig__get_device_id', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_domain_by_type', '_get_props_path', '_load_props', 'account', 'charset', 'get_token_path', 'is_paper', 'is_us', 'language', 'license', 'load_token', 'private_key', 'query_domains', 'quote_server_url', 'refresh_server_info', 'sdk_version', 'secret_key', 'server_url', 'should_token_refresh', 'sign_type', 'socket_host_port', 'store_token', 'tiger_id', 'tiger_public_key', 'timeout', 'timezone', 'token', 'token_refresh_duration']


In [15]:
# === Connect to Tiger Brokers ===
from tigeropen.common.util.signature_utils import read_private_key
from tigeropen.quote.quote_client import QuoteClient



YESTERDAY_FILE = r'./data/account_value_yesterday.txt'
def get_config(sandbox=False):
    cfg = TigerOpenClientConfig(sandbox_debug=sandbox)
    cfg.private_key = read_private_key(os.getenv("TIGER_PRIVATE_KEY_PEM"))  # Path to your private key file
    cfg.tiger_id = os.getenv("TIGER_ID")
    cfg.account = os.getenv("TIGER_PAPER_ACCOUNT")  # paper account
    cfg.language = Language.en_US
    return cfg

config = get_config(sandbox=False)
quote_client = QuoteClient(config)
trade_client = TradeClient(config)

# Check quote access (optional)
quote_client.grab_quote_permission()

trade_client = TradeClient(config)
account_summary = trade_client.get_assets()
print(account_summary)


# === Real Stop-Loss Check Using Live Account ===
daily_loss_threshold = 0.05

try:
    # Get current net liquidation value from Tiger account
    assets = trade_client.get_assets()
    current_value = float(assets[0].summary.net_liquidation)
    print(f"✅ Current account value: {current_value}")

    # Read yesterday’s value (default to 0.0 if file missing or empty)
    previous_value = 0.0
    if os.path.exists(YESTERDAY_FILE):
        with open(YESTERDAY_FILE, 'r') as f:
            content = f.read().strip()
            if content:
                previous_value = float(content)
            else:
                print("⚠️ Yesterday's file is empty. Assuming previous value = 0.0")
    else:
        print("📄 Yesterday's file not found. Assuming previous value = 0.0")

    # Calculate loss ratio (only if previous_value is positive)
    if previous_value > 0:
        daily_loss = (previous_value - current_value) / previous_value
        if daily_loss > daily_loss_threshold:
            logging.warning(f"[STOP LOSS TRIGGERED] Drop: {daily_loss:.2%}, "
                            f"Prev: {previous_value}, Now: {current_value}")
            print("❌ STOP: Live account drop exceeded threshold. No trades executed.")
            exit()

    # Save current value for next run
    with open(YESTERDAY_FILE, 'w') as f:
        f.write(str(current_value))
    print(f"✅ Live account loss check passed. Current value: {current_value}, Previous value: {previous_value}")

except Exception as e:
    logging.error(f"[STOP LOSS ERROR] Failed to check live account loss: {e}")
    print(f"Error checking live account loss: {e}")



[PortfolioAccount({'account': '21922446447411428', 'summary': Account({'accrued_cash': inf, 'accrued_dividend': inf, 'available_funds': inf, 'buying_power': 3691392.95, 'cash': 826362.65, 'currency': 'USD', 'cushion': inf, 'day_trades_remaining': inf, 'equity_with_loan': inf, 'excess_liquidity': inf, 'gross_position_value': inf, 'initial_margin_requirement': inf, 'maintenance_margin_requirement': inf, 'net_liquidation': 1003545.88, 'realized_pnl': 0.0, 'regt_equity': inf, 'regt_margin': inf, 'sma': inf, 'timestamp': None, 'unrealized_pnl': 3545.87}), 'segments': defaultdict(<class 'tigeropen.trade.domain.account.Account'>, {'S': SecuritySegment({'accrued_cash': 0.0, 'accrued_dividend': 0.0, 'available_funds': 820194.36, 'cash': 826362.65, 'equity_with_loan': 1003545.88, 'excess_liquidity': 936350.45, 'gross_position_value': 177183.23, 'initial_margin_requirement': 80697.64, 'leverage': 0.1766, 'maintenance_margin_requirement': 67195.43, 'net_liquidation': 1003545.88, 'regt_equity': inf

In [16]:
best_model

'ppo'

In [17]:
df_raw = YahooDownloader(
    start_date=TRAIN_START_DATE,
    end_date=TRADE_END_DATE,
    ticker_list=tickers
).fetch_data()

fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_vix=True,
    use_turbulence=True,
    user_defined_feature=False
)
processed = fe.preprocess_data(df_raw)

# Align to all dates and tickers
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(), processed['date'].max()).astype(str))
combination = list(itertools.product(list_date, list_ticker))
processed_full = pd.DataFrame(combination, columns=["date", "tic"]) \
    .merge(processed, on=["date", "tic"], how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])
processed_full = processed_full.fillna(0)

# Make sure 'date' column is in datetime format
processed_full['date'] = pd.to_datetime(processed_full['date'])

# Sort first for consistency
processed_full = processed_full.sort_values(by=['date', 'tic']).reset_index(drop=True)

# Assign the same index to all rows with the same date
processed_full.index = processed_full.groupby('date').ngroup()


stock_dimension = len(tickers)
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

buy_cost_list = sell_cost_list = [0.001] * stock_dimension
# Get live positions from Tiger Broker
positions = trade_client.get_positions()

# Make sure tickers are lowercase for matching
tickers_lower = [t.lower() for t in tickers]

# Build list of shares held per ticker in the correct order
num_stock_shares = [
    next((pos.quantity for pos in positions if hasattr(pos, "contract") and pos.contract.symbol.lower() == t), 0)
    for t in tickers_lower
]

print("Live num_stock_shares:", num_stock_shares)
assets = trade_client.get_assets()
account_summary = assets[0].summary
cash_balance = float(account_summary.cash)

env_kwargs = {
    "hmax": 100,
    "initial_amount": cash_balance,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

env = StockTradingEnv(df=processed_full, **env_kwargs)
obs, _ = env.reset()
initial_account_value = env.initial_amount
print(f"Initial account value: {initial_account_value}, Cash balance: {cash_balance}")

final_action = final_model.predict(obs, deterministic=True)[0]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (17034, 8)


[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (1001, 8)
Successfully added vix


Successfully added turbulence index
Stock Dimension: 17, State Space: 171
Live num_stock_shares: [0, 44, 72, 76, 0, 68, 0, 102, 0, 0, 0, 81, 84, 0, 0, 0, 81]
Initial account value: 826362.65, Cash balance: 826362.65


In [18]:
for ticker, shares in zip(tickers, num_stock_shares):
    print(f"{ticker}: {shares}")


aapl: 0
amd: 44
amzn: 72
cat: 76
crwd: 0
googl: 68
gs: 0
hd: 102
ibm: 0
intc: 0
meta: 0
msft: 81
nvda: 84
pypl: 0
t: 0
tsla: 0
v: 81


In [19]:
print(df_raw.date.max())

2025-06-23


In [20]:
from tigeropen.common.util.order_utils import (limit_order)           # Attached Order



# === Get account balance and positions ===
capital_limit = cash_balance * capital_limit_pct
logging.info(f"Cash balance: {cash_balance}, Capital allocation limit: {capital_limit:.2f}")
positions = trade_client.get_positions()
current_holdings = {pos.contract.symbol.lower(): pos.quantity for pos in positions if hasattr(pos, "contract") and pos.contract.symbol.lower() in tickers}
logging.info(f"Current holdings: {current_holdings}")

# === Execute Action Vector ===
def execute_trades(action_vector, tickers, trade_client, cash_balance):
    try:
        prices = {tic: yf.Ticker(tic).history(period='1d')['Close'].iloc[-1] for tic in tickers}
    except Exception as e:
        logging.error(f"Error fetching prices: {e}")
        return

    for i, a in enumerate(action_vector):
        symbol = tickers[i].upper()
        logging.info(f"Processing action for {symbol}: {a:.4f}")
        print(f"Processing action for {symbol}: {a:.4f}")
        price = prices.get(symbol.lower(), None)
        contract = trade_client.get_contracts(symbol=symbol)[0]
        if price is None:
            logging.warning(f"Price not available for {symbol}, skipping.")
            continue
        logging.info(f"Current price for {symbol}: ${price:.2f}")
        print(f"Current price for {symbol}: ${price:.2f}")
        shares = int(abs(a) * hmax)
        if shares == 0:
            continue

        action_type = None
        if a > 0.01:
            cost = shares * price
            if cost > capital_limit:
                # Adjust to maximum affordable shares
                shares = int(capital_limit / price)
                cost = shares * price
                print(f"💡 Adjusted BUY for {symbol}: {shares} shares within limit (${cost:.2f})")
                if shares == 0:
                    continue
            action_type = 'BUY'

        elif a < -0.01:
            held = current_holdings.get(symbol.lower(), 0)
            if shares > held:
                shares = held  # Sell only what you have
                print(f"💡 Adjusted SELL for {symbol}: only selling {shares} shares (held: {held})")
                if shares == 0:
                    continue
            action_type = 'SELL'

        else:
            continue

        logging.info(f"{action_type} {shares} shares of {symbol} at ${price:.2f}")
        print(f"{action_type} {shares} shares of {symbol} at ${price:.2f}")
        try:
            if action_type == 'BUY':
                price = round(price * 0.99, 2)
            elif action_type == 'SELL':
                price = round(price * 1.01, 2)
            stock_order = limit_order(
                account=config.account,
                contract=contract,
                action=action_type,
                limit_price=price,
                quantity=shares
            )
            trade_client.place_order(stock_order)
            print(stock_order)
        except Exception as e:
            logging.error(f"Order failed for {symbol}: {e}")
            print(f"Order failed for {symbol}: {e}")

# === Trade ===
execute_trades(final_action, tickers, trade_client, cash_balance)

# === Save today’s account value for next day ===
try:
    updated_assets = trade_client.get_assets()
    latest_value = float(updated_assets[0].summary.net_liquidation)
    with open(YESTERDAY_FILE, 'w') as f:
        f.write(str(latest_value))
    logging.info(f"Saved latest account value for stop-loss: {latest_value}")
    print(f"Saved latest account value for stop-loss: {latest_value}")
except Exception as e:
    logging.error(f"Failed to save account value for stop-loss check: {e}")
    print(f"Failed to save account value for stop-loss check: {e}")

logging.info("SMART trade execution complete.")
print("SMART trade execution complete.")

Processing action for AAPL: 0.1178
Current price for AAPL: $201.50
BUY 11 shares of AAPL at $201.50
Order({'account': '21922446447411428', 'id': 39566642674010112, 'order_id': None, 'parent_id': None, 'order_time': None, 'reason': None, 'trade_time': None, 'action': 'BUY', 'quantity': 11, 'filled': 0, 'avg_fill_price': 0, 'commission': None, 'realized_pnl': None, 'trail_stop_price': None, 'limit_price': np.float64(199.48), 'aux_price': None, 'trailing_percent': None, 'percent_offset': None, 'order_type': 'LMT', 'time_in_force': None, 'outside_rth': None, 'order_legs': None, 'algo_params': None, 'algo_strategy': None, 'secret_key': None, 'liquidation': None, 'discount': None, 'attr_desc': None, 'source': None, 'adjust_limit': None, 'sub_ids': [], 'user_mark': None, 'update_time': None, 'expire_time': None, 'can_modify': None, 'external_id': None, 'combo_type': None, 'combo_type_desc': None, 'is_open': None, 'contract_legs': None, 'filled_scale': None, 'total_cash_amount': None, 'filled_

In [21]:
from datetime import datetime
import os
# === Create folder if it doesn't exist ===
os.makedirs("transaction_actions", exist_ok=True)

# === Prepare log content ===
log_lines = []
today_str = datetime.now().strftime('%Y-%m-%d')
log_file = f"transaction_actions/transaction_{today_str}.txt"

try:
    # ✅ Current Holdings
    log_lines.append(f"📅 Date: {today_str}\n")
    log_lines.append("✅ Current Holdings:")
    positions = trade_client.get_positions()
    for pos in positions:
        if pos.quantity > 0 and hasattr(pos, "contract") and hasattr(pos.contract, "symbol"):
            log_lines.append(f" - {pos.contract.symbol}: {pos.quantity} shares")
    print("Current Holdings Completed")
    # ⏳ Pending Orders
    log_lines.append("\n⏳ Pending Orders:")
    open_orders = trade_client.get_open_orders()
    for order in open_orders:
        if hasattr(order, 'contract') and hasattr(order.contract, 'symbol') and hasattr(order, 'action') and hasattr(order, 'order_id'):
            price_info = getattr(order, 'limit_price', 'N/A')
            status = getattr(order, 'status', 'Unknown')
            log_lines.append(f"- {order.contract.symbol}: {order.action} order (Limit: {price_info}, Qty: {order.quantity}) [Status: {status}, ID: {order.order_id}]")
    if not open_orders:
        log_lines.append(" - None")
    print("Pending Orders Completed")
    # 💼 Executed Orders (today only)
    log_lines.append("\n💼 Executed Orders (Today):")
    all_orders = trade_client.get_orders()
    for order in all_orders:
        filled_time = getattr(order, 'filled_time', None)
        if filled_time and filled_time.strftime('%Y-%m-%d') == today_str:
            log_lines.append(f"- {order.contract.symbol}: {order.action} order filled (Price: {getattr(order, 'filled_avg_price', 'N/A')}, Qty: {order.quantity}, ID: {order.order_id})")
    if not all_orders:
        log_lines.append(" - None")
except Exception as e:
    log_lines.append(f"\n⚠️ Error retrieving open orders or positions: {e}")

# === Save to file ===
try:
    with open(log_file, 'w') as f:
        f.write('\n'.join(log_lines))
    print(f"📄 Transaction log saved: {log_file}")
except Exception as e:
    print(f"❌ Failed to write transaction log: {e}")

Current Holdings Completed
Pending Orders Completed
📄 Transaction log saved: transaction_actions/transaction_2025-06-24.txt
